# ELYZA-tasks-100 による性能評価

In [ ]:
!pip install accelerate datasets jsonlines sentencepiece

## ELYZA-tasks-100 データセットのダウンロード

モデルのダウンロードには 🤗datasets ライブラリを用いる。

データは `/content/dataset.jsonl` に保存する。

In [ ]:
import json
from pathlib import Path

from datasets import load_dataset

# Load the dataset
ds = load_dataset("elyza/ELYZA-tasks-100")

# Function to convert dataset to JSONL format and print
def dataset_to_jsonl(dataset, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        for entry in dataset:
            # Construct JSON object
            json_obj = {
                "input_text": entry['input'],
                "output_text": entry['output'],
                "eval_aspect": entry['eval_aspect']
            }

            # Write JSON object to file in JSONL format
            json_str = json.dumps(json_obj, ensure_ascii=False)
            file.write(json_str + '\n')

# Convert and write the dataset to a file in JSONL format
path_jsonl = Path("/content/dataset.jsonl")

if (path_jsonl.parent is not None) and (not path_jsonl.parent.exists()):
    path_jsonl.parent.mkdir(parents=True, exist_ok=True)

dataset_to_jsonl(ds["test"], path_jsonl)


In [ ]:
!pip install hf_transfer
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download TheBloke/japanese-stablelm-instruct-beta-70B-GGUF \
    --local-dir . \
    --local-dir-use-symlinks False \
    --include='*Q4_K_M*gguf*'

downloading https://huggingface.co/TheBloke/japanese-stablelm-instruct-beta-70B-GGUF/resolve/3228f382ca3a63b38a138d136686e41d8f127173/japanese-stablelm-instruct-beta-70b.Q4_K_M.gguf to /root/.cache/huggingface/hub/tmpcr1clncg
(…)e-stablelm-instruct-beta-70b.Q4_K_M.gguf: 100% 41.4G/41.4G [02:10<00:00, 317MB/s]
/content


## 🚬Smoke test

`llama.cpp` は、デフォルトではメモリマップされたファイルを使用する。`--no-mmap` を有効化する。

[llama.cpp](https://github.com/ggerganov/llama.cpp/issues/2589#issuecomment-1693312217) を参照。

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python

In [ ]:
from llama_cpp import Llama

llm = Llama(
    model_path="./japanese-stablelm-instruct-beta-70b.Q4_K_M.gguf",
    n_gpu_layers=65,
    use_mmap=False,
    seed=42,
    n_ctx=2048,
    n_threads=11,
)

AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 1 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [ ]:
# Simple inference example
# Question: https://www.reddit.com/r/LocalLLaMA/comments/13zz8y5/what_questions_do_you_ask_llms_to_check_their/?utm_source=share&utm_medium=web3x&utm_name=web3xcss&utm_term=1&utm_content=share_button
prompt = """<s>[INST] <<SYS>>
あなたは役立つアシスタントです。
<<SYS>>

蚊を殺すことは、社会通念上許容される行為ですか？ [/INST] """
output = llm(
  prompt,
  temperature=0.7,
  echo=False,
  stop=["</s>"], # 🚨Not necessarily correct for this specific model! Please check before using.
  repeat_penalty=1.1,
  seed=42,
  max_tokens=-1,
)
print(output["choices"][0]["text"])

 そうです！蚊を殺すのは完全に合法です。


## 評価

ELYZA-tasks-100 データセットを用いて、指定したモデルの評価を実行する。

### プロンプトについて

モデルに回答を生成させるときのプロンプトは、各モデルのモデルカード (🤗HuggingFace の README ページ) に記載されているプロンプトを使用する。

> 📒 **ノート**
>
> この方法は、質問を直接入力する場合や、全モデルで共通のプロンプトを用いる場合に比べて公平性が劣る。
> しかし、実利用時は各モデルが最もよい性能を発揮するように、各モデルのプロンプトをチューニングすることが想定される。
> モデルカード記載のプロンプトをチューニング済みのプロンプトと見なすことで、実利用時の性能を比較することを狙いとする。

### ハイパーパラメータについて

ハイパーパラメータは 🤗HuggingFace の Code Snippet のとおりとする（公開者が指定したハイパーパラメータを、チューン済みとみなす）。
ただし、長文の回答を要求する設問もあるため、`max_new_tokens` は `1024` に統一する。

In [ ]:
import jsonlines

def generate_text_with_llama(input_text):
    # Create a chat completion using the Llama object
    prompt = f"""<s>[INST] <<SYS>>
あなたは役立つアシスタントです。
<<SYS>>

{input_text} [/INST] """
    output = llm(
        prompt,
        echo=False,
        stop=["</s>"], # 🚨Not necessarily correct for this specific model! Please check before using.
        temperature=0.99,
        top_p=0.95,
        seed=42,
        max_tokens=-1,
    )
    return output["choices"][0]["text"]

def process_dataset(input_file, output_file):
    # Read dataset and generate predictions
    with jsonlines.open(input_file) as reader, jsonlines.open(output_file, mode='w') as writer:
        for obj in reader:
            prompt = obj['input_text']
            generated_text = generate_text_with_llama(prompt)

            print(f"==============================")
            print(f"Q. {prompt}")
            print(f"A. {generated_text}")
            print(f"")

            writer.write({"pred": generated_text})

# Process the dataset
input_dataset = '/content/dataset.jsonl'
output_predictions = '/content/preds.jsonl'
process_dataset(input_dataset, output_predictions)


Llama.generate: prefix-match hit


Q. 仕事の熱意を取り戻すためのアイデアを5つ挙げてください。
A.  - あなたが情熱を注いでいることを思い出してください
- 休みを入れる
- 職場の友人に相談する
- 上司に相談する
- 新しい仕事を探す



Llama.generate: prefix-match hit


Q. クマが海辺に行ってアザラシと友達になり、最終的には家に帰るというプロットの短編小説を書いてください。
A.  クマは海辺に行きました。そこでアザラシに出会い、一緒に遊び始めました。クマとアザラシはずっと一緒に過ごして楽しかったのですが、最後にクマは家に帰ることにしました。



Llama.generate: prefix-match hit


...

Q. 漢字に対応する意味を選び、対応する数字とアルファベットのペアを解答してください。

1. 立派　A. すばらしい
2. 開拓　B. ある事柄について知っている内容。
3. 知識　C. 物事に味わいや価値を加える
4. 賞賛　D. 新しい領域を切り開く
5. 味付け　E. 高い評価を与える
A.  A.立派な B.知識 C. 開拓 D. 賞賛 E. 味付け



Llama.generate: prefix-match hit


Q. 次の物語の展開を予想してみましょう。

ある日、主人公のもとに不思議な手紙が届きました。手紙には「今夜の満月に、森の奥深くにある洞窟に来てください。あなたを待っています。」と書かれていました。主人公はその手紙に従い、夜中に洞窟の入り口にたどり着きました。中に入ると、謎めいた人物と出会い……。
A.  主人公は、手紙の主である謎めいた人物に導かれ、魔法のような世界への扉を開ける鍵の仕組みを知ることになるかもしれません。



Llama.generate: prefix-match hit


Q. 次の文は1つの物語の文の順番をシャッフルしたものです。適切な順に並び替えてください。

1. 「果物だけではなく、人生も同じです。協力しなくては、全てをこなすことはできません。」
2. 老人は、広い畑で色々な果物を育てていました。
3. ある日、孫が老人に『こんなに広い畑で、全ての果物を一人で育てるのは大変そうですね』と尋ねました。
4. 昔、あるところに老人と孫がいました。
5. 老人は微笑みながら次のように答えました。

A.  正しい順番は4、2、3、5です。

